In [181]:
#The output of this section is the CSV files with the data to be handle by the model
CREATE_CSV_FILES = True

In [182]:
SPEECH_TRAIN_CSV_FILE = "speach_recognition_train.csv"
SPEECH_TEST_CSV_FILE = "speech_recognition_test.csv"

In [236]:
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
from matplotlib import cm
import pandas as pd
import librosa
import zipfile as zf
import csv
import os
from sklearn import metrics
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
from sklearn import tree
# import zipfile as zf
from sklearn import svm
from sklearn.svm import SVC
import pickle

In [237]:
def extractWavFeatures(soundFilesFolder, csvFileName):
    print("The features of the files in the folder "+soundFilesFolder+" will be saved to "+csvFileName)
    header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
#     for i in range(1, 61):
#         header += f' mfcc{i}'
    header += ' label'
    header = header.split()
#     print('CSV Header: ', header)
    file = open(csvFileName, 'w', newline='')
    #with file:
    writer = csv.writer(file)
    writer.writerow(header)
    for filename in os.listdir(soundFilesFolder):
        if filename.endswith('.wav'):
            number = f'{soundFilesFolder}/{filename}'
            y, sr = librosa.load(number, mono=True, duration = 30)
            # remove leading and trailing silence
            y, index = librosa.effects.trim(y)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
#             mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 60)
            to_append = f'{filename} {np.mean(rmse)} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
#             for e in mfcc:
#                 to_append += f' {np.mean(e)}'
            writer.writerow(to_append.split())
    file.close()
    print("End of extractWavFeatures")

In [238]:
if (CREATE_CSV_FILES == True):
    extractWavFeatures("Audio_Data/Speech_Recognition_Train_Data_final", SPEECH_TRAIN_CSV_FILE)
    extractWavFeatures("Audio_Data/Speach_test_data", SPEECH_TEST_CSV_FILE)
    print("CSV files are created")
else:
    print("CSV files creation is skipped")

The features of the files in the folder Audio_Data/Speech_Recognition_Train_Data_final will be saved to speach_recognition_train.csv
End of extractWavFeatures
The features of the files in the folder Audio_Data/Speach_test_data will be saved to speech_recognition_test.csv
End of extractWavFeatures
CSV files are created


In [239]:
#Reading a dataset and convert file name to corresbonding number
def speech_preProcessData(csvFileName):
#     header_name_list = ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'mfcc21', 'mfcc22', 'mfcc23', 'mfcc24', 'mfcc25', 'mfcc26', 'mfcc27', 'mfcc28', 'mfcc29', 'mfcc30', 'mfcc31', 'mfcc32', 'mfcc33', 'mfcc34', 'mfcc35', 'mfcc36', 'mfcc37', 'mfcc38', 'mfcc39', 'mfcc40', 'label']
    print(csvFileName+ " will be preprocessed")
    data =  pd.read_csv(csvFileName)
#     data = pd.read_csv(csvFileName, skiprows=[1, 50]
    # we have 2 sentances: 
    # 0: open the door
    # 1: close the door
    filenameArray = data['filename'] 
    speechTypeArray = []
#     print(filenameArray)
    for filename in filenameArray:
        #print(speaker)
        if "open_the_door" in filename:
            speaker = 0
        else: 
            speaker = 1
#         print(speaker)
        speechTypeArray.append(speaker)
    data['number'] = speechTypeArray
    #Dropping unnecessary columns
    data = data.drop(['filename'],axis=1)
    data = data.drop(['label'],axis=1)
#     data = data.drop(['chroma_stft'],axis=1)
#     data.shape
#     print("Preprocessing is finished")
#     print(data[['filename', 'number']])
#     print(data)
    return data

In [240]:
speechtrainData = speech_preProcessData(SPEECH_TRAIN_CSV_FILE)
speechtestData = speech_preProcessData(SPEECH_TEST_CSV_FILE)

speach_recognition_train.csv will be preprocessed
speech_recognition_test.csv will be preprocessed


In [241]:
speechtrainData

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,number
0,0.045838,0.324215,1158.867914,1396.735259,2292.729240,0.049880,0
1,0.025397,0.448211,2315.191869,2825.016168,5660.115774,0.088334,1
2,0.029656,0.462529,2252.689010,2801.919295,5501.109248,0.072039,1
3,0.015112,0.294097,1264.958471,1501.696181,2399.696045,0.056331,1
4,0.039543,0.364756,1195.501657,1441.521266,2293.134491,0.052418,0
...,...,...,...,...,...,...,...
137,0.035723,0.530913,2264.746003,2785.203359,5571.524048,0.075823,1
138,0.009834,0.346504,1321.168317,1710.770952,2618.676758,0.052479,1
139,0.023293,0.420064,1385.335041,1489.298766,2711.007366,0.077860,1
140,0.032848,0.444131,1172.923283,1470.429012,2282.157629,0.049333,0


In [242]:
X_train_speech = np.array(speechtrainData.iloc[:, :-1], dtype = float)
y_train_speech = speechtrainData.iloc[:, -1]
X_test_speech = np.array(speechtestData.iloc[:, :-1], dtype = float)
y_test_speech = speechtestData.iloc[:, -1]

print("Y from training data:", y_train_speech.shape)
print("Y from test data:", y_test_speech.shape)

Y from training data: (142,)
Y from test data: (18,)


In [243]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()
X_train_speech = scaler.fit_transform( X_train_speech )
X_test_speech = scaler.transform( X_test_speech )
print("X from training data", X_train_speech.shape)
print("X from test data", X_test_speech.shape)

X from training data (142, 6)
X from test data (18, 6)


In [244]:
Dtr_speech_classifier = DecisionTreeClassifier()
Dtr_speech_classifier.fit(X_train_speech, y_train_speech)

DecisionTreeClassifier()

In [245]:
Svm_speech_classifier = SVC()  
Svm_speech_classifier.fit(X_train_speech, y_train_speech)  

SVC()

In [246]:
Dtr_speech_classifier_prediciton = Dtr_speech_classifier.predict(X_test_speech)
Dtr_speech_classifier_prediciton

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0])

In [247]:
Svm_speech_classifier_pred = Svm_speech_classifier.predict(X_test_speech)
Svm_speech_classifier_pred

array([1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0])

In [248]:
print("Accuracy_speech_classifier_SVM:",metrics.accuracy_score(y_test_speech, Svm_speech_classifier_pred))

Accuracy_speech_classifier_SVM: 0.5555555555555556


In [249]:
print("Accuracy_speech_classifier_Dtr:",metrics.accuracy_score(y_test_speech, Dtr_speech_classifier_prediciton))

Accuracy_speech_classifier_Dtr: 0.5555555555555556


In [250]:
def extract_features_from_input(input_audio):
    extracted_features = []
    y, sr = librosa.load(input_audio, mono=True)
    y, index = librosa.effects.trim(y)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)

#     mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 60)
    extracted_features.extend([np.mean(rmse), np.mean(chroma_stft), np.mean(spec_cent),np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)])
#     for e in mfcc:
#         extracted_features.append(np.mean(e))
    return np.array(extracted_features)

In [268]:
input_audio = 'Audio_Data/open_test1.wav'
audio_features = extract_features_from_input(input_audio)
audio_features = audio_features.reshape(1,6)

In [269]:
prediction = Dtr_speech_classifier.predict(np.array(audio_features))
speakers_list = [(0, 'Open'), (1, 'other')]
for iterator, speech in enumerate(speakers_list):
    if prediction[0] == speech[0]:
        print("voice of " + speech[1])

voice of Open


In [257]:
# save the model to disk
import pickle
filename = 'speech_classifier_final.sav'
pickle.dump(Dtr_speech_classifier, open(filename, 'wb'))

In [258]:
prediction = Svm_speech_classifier.predict(np.array(audio_features))
speakers_list = [(0, 'Open'), (1, 'other')]
for iterator, speech in enumerate(speakers_list):
    if prediction[0] == speech[0]:
        print("voice of " + speech[1])

voice of Open


RandomForestClassifier()

In [234]:
# prediction on test set
y_pred=clf.predict(X_test_speech)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test_speech, y_pred))

Accuracy: 0.6666666666666666


In [263]:
prediction = clf.predict(np.array(audio_features))
speakers_list = [(0, 'Open'), (1, 'other')]
for iterator, speech in enumerate(speakers_list):
    if prediction[0] == speech[0]:
        print("voice of " + speech[1])

voice of other


In [59]:
filename = 'Dtr_speech_classifier.sav'
pickle.dump(Dtr_speech_classifier, open(filename, 'wb'))

## 